# [NTDS'19] tutorial 2: build a graph from an edge list
[ntds'19]: https://github.com/mdeff/ntds_2019

[Benjamin Ricaud](https://people.epfl.ch/benjamin.ricaud), [EPFL LTS2](https://lts2.epfl.ch)

* Dataset: [Open Tree of Life](https://tree.opentreeoflife.org)
* Tools: [pandas](https://pandas.pydata.org), [numpy](http://www.numpy.org), [networkx](https://networkx.github.io), [gephi](https://gephi.org/)

## Tools

By convention, the first lines of code are always about importing the packages we'll use.

In [1]:
import pandas as pd
import numpy as np
import networkx as nx

Tutorials on pandas can be found at:
* <https://pandas.pydata.org/pandas-docs/stable/10min.html>
* <https://pandas.pydata.org/pandas-docs/stable/tutorials.html>

Tutorials on numpy can be found at:
* <https://docs.scipy.org/doc/numpy/user/quickstart.html>
* <http://www.scipy-lectures.org/intro/numpy/index.html>
* <http://www.scipy-lectures.org/advanced/advanced_numpy/index.html>

A tutorial on networkx can be found at:
* <https://networkx.github.io/documentation/stable/tutorial.html>

## Import the data

We will play with a excerpt of the Tree of Life, that can be found together with this notebook. This dataset is reduced to the first 1000 taxons (starting from the root node). The full version is available here: [Open Tree of Life](https://tree.opentreeoflife.org/about/taxonomy-version/ott3.0).

![Public domain, https://en.wikipedia.org/wiki/File:Phylogenetic_tree.svg](https://upload.wikimedia.org/wikipedia/commons/thumb/7/70/Phylogenetic_tree.svg/800px-Phylogenetic_tree.svg.png)

In [35]:
tree_of_life = pd.read_csv('data/taxonomy_small.tsv', sep='\t\|\t?', encoding='utf-8', engine='python')

If you do not remember the details of a function:

In [4]:
pd.read_csv?

Signature: pd.read_csv(filepath_or_buffer, sep=',', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, iterator=False, chunksize=None, compression='infer', thousands=None, decimal=b'.', lineterminator=None, quotechar='"', quoting=0, escapechar=None, comment=None, encoding=None, dialect=None, tupleize_cols=None, error_bad_lines=True, warn_bad_lines=True, skipfooter=0, doublequote=True, delim_whitespace=False, low_memory=True, memory_map=False, float_precision=None)
Docstring:
Read CSV (comma-separated) file into DataFrame

Also supports optionally iterating or breaking of the file
into chunks.



For more info on the separator, see [regex](https://docs.python.org/3.6/library/re.html).

Now, what is the object `tree_of_life`? It is a Pandas DataFrame.

In [ ]:
tree_of_life

The description of the entries is given here:
https://github.com/OpenTreeOfLife/reference-taxonomy/wiki/Interim-taxonomy-file-format

## Explore the table

In [36]:
tree_of_life.columns

Index(['uid', 'parent_uid', 'name', 'rank', 'sourceinfo', 'uniqname', 'flags',
       'Unnamed: 7'],
      dtype='object')

Let us drop some columns.

In [37]:
tree_of_life = tree_of_life.drop(columns=['sourceinfo', 'uniqname', 'flags','Unnamed: 7'])

In [38]:
tree_of_life.head()

,uid,parent_uid,name,rank
0,805080,NaN,life,no rank
1,93302,805080.0,cellular organisms,no rank
2,996421,93302.0,Archaea,domain
3,5246114,996421.0,Marine Hydrothermal Vent Group 1(MHVG-1),no rank - terminal
4,102415,996421.0,Thaumarchaeota,phylum


Pandas infered the type of values inside each column (int, float, string and string). The parent_uid column has float values because there was a missing value, converted to `NaN`

In [6]:
print(tree_of_life['uid'].dtype, tree_of_life.parent_uid.dtype)

int64 float64


How to access individual values.

In [7]:
tree_of_life.iloc[0, 2]

'life'

In [8]:
tree_of_life.loc[0, 'name']

'life'

**Exercise**: Guess the output of the following line:

In [9]:
#tree_of_life.uid[0] == tree_of_life.parent_uid[1]

Ordering the data.

In [10]:
tree_of_life.sort_values(by='name').head()

,uid,parent_uid,name,rank
297,5246638,102415.0,AB64A-17,no rank - terminal
293,5246632,102415.0,AK31,no rank - terminal
298,5246637,102415.0,AK56,no rank - terminal
202,5246635,102415.0,AK59,no rank - terminal
204,5246636,102415.0,AK8,no rank - terminal


 *Remark:* Some functions do not change the dataframe (option `inline=False` by default).

In [11]:
tree_of_life.head()

,uid,parent_uid,name,rank
0,805080,NaN,life,no rank
1,93302,805080.0,cellular organisms,no rank
2,996421,93302.0,Archaea,domain
3,5246114,996421.0,Marine Hydrothermal Vent Group 1(MHVG-1),no rank - terminal
4,102415,996421.0,Thaumarchaeota,phylum


## Operation on the columns

Unique values, useful for categories:

In [ ]:
tree_of_life['rank'].unique()

Selecting only one category.

In [ ]:
tree_of_life[tree_of_life['rank'] == 'species'].head()

How many species do we have?

In [ ]:
len(tree_of_life[tree_of_life['rank'] == 'species'])

In [ ]:
tree_of_life['rank'].value_counts()

**Exercise:** Display the entry with name 'Archaea', then display the entry of its parent.

In [ ]:
# Your code here.
parent_archaea = tree_of_life[tree_of_life.name == 'Archaea'].parent_uid.values[0]

In [ ]:
tree_of_life[tree_of_life.uid == parent_archaea]

## Building the graph

Let us build the adjacency matrix of the graph. For that we need to reorganize the data. First we separate the nodes and their properties from the edges.

In [39]:
nodes = tree_of_life[['uid', 'name','rank']]
edges = tree_of_life[['uid', 'parent_uid']]

When using an adjacency matrix, nodes are indexed by their row or column number and not by a `uid`. Let us create a new index for the nodes.

In [32]:
# Create a column for node index.
nodes.reset_index(level=0, inplace=True)

In [28]:
nodes = nodes.rename(columns={'index':'node_idx'})
nodes.head()

,node_idx,uid,name,rank
0,0,805080,life,no rank
1,1,93302,cellular organisms,no rank
2,2,996421,Archaea,domain
3,3,5246114,Marine Hydrothermal Vent Group 1(MHVG-1),no rank - terminal
4,4,102415,Thaumarchaeota,phylum


In [16]:
# Create a conversion table from uid to node index.
uid2idx = nodes[['node_idx', 'uid']]
uid2idx = uid2idx.set_index('uid')
uid2idx.head()

,node_idx
uid,
805080,0
93302,1
996421,2
5246114,3
102415,4


In [17]:
edges.head()

,uid,parent_uid
0,805080,NaN
1,93302,805080.0
2,996421,93302.0
3,5246114,996421.0
4,102415,996421.0


Now we are ready to use yet another powerful function of Pandas. Those familiar with SQL will recognize it: the `join` function.

In [18]:
# Add a new column, matching the uid with the node_idx.
edges = edges.join(uid2idx, on='uid')

In [19]:
# Do the same with the parent_uid.
edges = edges.join(uid2idx, on='parent_uid', rsuffix='_parent')

In [20]:
# Drop the uids.
edges_renumbered = edges.drop(columns=['uid','parent_uid'])

The `edges_renumbered` table is a list of renumbered edges connecting each node to its parent.

In [21]:
edges_renumbered.head()

,node_idx,node_idx_parent
0,0,NaN
1,1,0.0
2,2,1.0
3,3,2.0
4,4,2.0


## Building the (weighted) adjacency matrix

We will use numpy to build this matrix. Note that we don't have edge weights here, so our graph is going to be unweighted.

In [24]:
nodes

,node_idx,uid,name,rank
0,0,805080,life,no rank
1,1,93302,cellular organisms,no rank
2,2,996421,Archaea,domain
3,3,5246114,Marine Hydrothermal Vent Group 1(MHVG-1),no rank - terminal
4,4,102415,Thaumarchaeota,phylum
...,...,...,...,...
994,994,5571591,uncultured marine thaumarchaeote KM3_175_A05,species
995,995,5571756,uncultured marine thaumarchaeote KM3_46_E07,species
996,996,5571888,uncultured marine thaumarchaeote KM3_02_A10,species
997,997,5205131,thaumarchaeote enrichment culture clone Ec.FBa...,species


In [22]:
n_nodes = len(nodes)
adjacency = np.zeros((n_nodes, n_nodes), dtype=int)

In [23]:
edges

,uid,parent_uid,node_idx,node_idx_parent
0,805080,NaN,0,NaN
1,93302,805080.0,1,0.0
2,996421,93302.0,2,1.0
3,5246114,996421.0,3,2.0
4,102415,996421.0,4,2.0
...,...,...,...,...
994,5571591,102415.0,994,4.0
995,5571756,102415.0,995,4.0
996,5571888,102415.0,996,4.0
997,5205131,102415.0,997,4.0


In [ ]:
for idx, row in edges.iterrows():
    if np.isnan(row.node_idx_parent):
        continue
    i, j = int(row.node_idx), int(row.node_idx_parent)
    adjacency[i, j] = 1  # weight
    adjacency[j, i] = 1  # weight to obtain an undirected network

In [ ]:
adjacency[:15, :15]

Congratulations, you have built the adjacency matrix!

## The graph

In [ ]:
# A simple command to create the graph from the adjacency matrix.
graph = nx.from_numpy_array(adjacency)

In addition, let us add some attributes to the nodes:

In [ ]:
node_props = nodes.to_dict()

In [ ]:
node_props.keys()

In [ ]:
for key in node_props:
    # print(key, node_props[key])
    nx.set_node_attributes(graph, node_props[key], key)

Let us check if it is correctly recorded:

In [ ]:
graph.node[1]

**Exercise 1:**
Build the graph directly from the `edges` table (without using the adjacency matrix).

In [ ]:
g = nx.from_pandas_edgelist(edges, source='node_idx_parent', target='node_idx')

**Exercise 2:**
Build the graph from the initial `tree_of_life` table by directly iterating over the rows of this table (without building the adjacency matrix).

In [ ]:
# Your code here.
gr = nx.Graph()

In [ ]:
tree_of_life

In [ ]:
for idx, row in tree_of_life.iterrows():
    if np.isnan(row[1]):
        continue
    gr.add_edge(row[1], row[0], attr_dict=row[2:].to_dict())

**Exercise 3:**
Get the adjacency matrix with `nx.adjacency_matrix(graph)` and compare it with what we obtained previously.

In [ ]:
# Your code here.
a, b = nx.adjacency_matrix(graph), nx.adjacency_matrix(gr)

In [ ]:
(a!=b).nnz==0

In [ ]:
import scipy

In [ ]:
a=scipy.sparse.csr_matrix([[0,1],[1,0]])
b=scipy.sparse.csr_matrix([[0,1],[1,0]])
(a!=b).nnz==0   


## Graph visualization

To conclude, let us visualize the graph. We will use the python module networkx.

The following line is a [magic command](https://ipython.readthedocs.io/en/stable/interactive/magics.html). It enables plotting inside the notebook.

In [ ]:
%matplotlib inline

You may also try `%matplotlib notebook` for a zoomable version of plots.

Draw the graph with two different [layout algorithms](https://en.wikipedia.org/wiki/Graph_drawing#Layout_methods).

In [ ]:
nx.draw_spectral(graph)

In [ ]:
nx.draw_spring(graph)

Save the graph to disk in the `gexf` format, readable by gephi and other tools that manipulate graphs. You may now explore the graph using [gephi](https://gephi.org/) and compare the visualizations.

In [ ]:
nx.write_gexf(graph, 'tree_of_life.gexf')